In [1]:
#Import required libraries
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#List the locations of input data and directories
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
#Unzip the train and test data

!unzip -o ../input/facial-keypoints-detection/test.zip
!unzip -o ../input/facial-keypoints-detection/training.zip

In [4]:
#Read csv data into a DataFrame

train_data = pd.read_csv('../input/khoa-train/training.csv')
test_data = pd.read_csv('../input/khoa123/test.csv')
id_lookup = pd.read_csv('../input/look-up/IdLookupTable.csv')

In [5]:
#Number of data samples in train and test set

len(train_data),len(test_data)

In [6]:
#Show sample training data 

train_data = train_data[['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x', 'mouth_right_corner_y', 'Image']]


In [7]:
#Show sample test data
test_data.head()

In [8]:
#Show sample data from id_lookup file
id_lookup.head()

In [9]:
train_data.isnull().sum()

In [10]:
#Drop null values
train_data_clean = train_data.dropna()

In [11]:
#Create list of feature and labels
images = []
labels = []

for i, sample in train_data_clean.iterrows():
    #Features
    img = np.array((sample['Image'].split(' ')), dtype=float)
    img = np.reshape(img, (96, 96, 1))
    images.append(img)
    #Labels
    labels.append(sample[0:8])  

In [12]:
# Define function to plot images and markers
def plot_image(img_id, img, lab, axis):
    axis.imshow(img[img_id], cmap='gray')
    axis.scatter(lab[img_id][0:30:2], lab[img_id][1:30:2], marker='x', c = 'red')

# Plot 5 sample images
fig = plt.figure(figsize=(15,8))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])    
    plot_image(i, images, labels, ax)

In [13]:
# Import all building blocks to build a CNN using Keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dropout, MaxPooling2D
from keras import Input
#from keras.layers.advanced_activations import LeakyReLU

In [14]:
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=(96,96,1), activation='sigmoid'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
        
model.add(Conv2D(128, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
          
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding='same', activation='sigmoid'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())



model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(8))
model.summary()

In [15]:
#Compile model with optimizer and loss function
import tensorflow as tf
from tensorflow import keras
model.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss='mean_squared_error',
#              metrics=[tf.keras.metrics.RootMeanSquaredError()])
              metrics=['accuracy',tf.keras.metrics.RootMeanSquaredError()])

In [16]:
# Converting lists to array in order to be compatable with model.fit()
img = np.array(images, dtype=float)
label = np.array(labels, dtype=float)


In [17]:
#Fit the model to training data
history = model.fit(img, label, epochs=250, validation_split=0.3)
model.save('my_model.h5')

In [18]:
history.history.keys()

In [19]:
fig, (ax1, ax2,) = plt.subplots(1, 2,  figsize=(15,5))
ax1.plot(history.history['root_mean_squared_error'])
ax1.plot(history.history['val_root_mean_squared_error'])
ax1.set_title('RMS')
ax1.set(xlabel='epoch', ylabel='rms')
ax1.legend(['train', 'val'])

ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])
ax2.set_title('độ chính xác')
ax1.set(xlabel='epoch', ylabel='rms')
ax2.legend(['train', 'val'])

# ax3.plot(history.history['loss'])
# ax3.plot(history.history['val_loss'])
# ax3.set_title('Loss')
# ax1.set(xlabel='epoch', ylabel='rms')
# ax3.legend(['train', 'val'])

In [20]:
#Create test data
test_img = []

for i, sample in test_data.iterrows():
    #Features
    img = np.array((sample['Image'].split(' ')), dtype=float)
    img = np.reshape(img, (96, 96, 1))
#     print(img.shape)
    test_img.append(img) 
test_img = np.array(test_img, dtype=float)

In [21]:
#Predict on test set
predict = model.predict(test_img)
predict

In [22]:
fig = plt.figure(figsize=(20,6))
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    plot_image(i+1, test_images, predict, ax)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,6))
for i in range(3):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])
    plot_image(i, test_images, predict, ax)
plt.show()

In [ ]:
model.save('my_model.h5')

In [ ]:
import cv2
import dlib
import argparse
import time
import numpy as np
from keras.models import load_model

from IPython.display import Image
import os
!ls ../input/

In [27]:
path ="../input/khoalovely/Untitled.png"
image = tf.keras.preprocessing.image.load_img(path, color_mode='grayscale', target_size=(96,96,1),  interpolation="nearest")
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr], dtype=float)  # Convert single image to a batch.
print(input_arr.shape)
prediction = model.predict(input_arr)
print(prediction)
fig = plt.figure(figsize=(50,6))
for i in range(1):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])
    plot_image(i, input_arr, prediction, ax)
plt.show()

In [23]:
#hàm tính tỉ lệ
from math import sqrt
def plot_image(img_id, img, lab, axis):
    ti_le_vang = 1.618
    axis.imshow(img[img_id], cmap='gray')
    axis.scatter(lab[img_id][0:8:2], lab[img_id][1:8:2], marker='o', c = 'blue')
    kc_mat = sqrt(abs(lab[img_id][0] - lab[img_id][2])  + abs(lab[img_id][1] - lab[img_id][3]) )
    kc_mieng = sqrt(abs(lab[img_id][4] - lab[img_id][6]) + abs(lab[img_id][5] - lab[img_id][7]))
    axis.set_xlabel(f'phần trăm tỉ lệ vàng: {(kc_mat / kc_mieng) * 100/ ti_le_vang:.2f}%')
# Plot 20 sample images
fig = plt.figure(figsize=(20,10))
for i in range(20):
    ax = fig.add_subplot(4, 5, i+1, xticks=[], yticks=[])    
    plot_image(i, images, labels, ax)